## COVID_PROJECT starts from here

In [0]:
# Importing required libraries. The Plotly library was not working so did most of the visualization on Colab. Implemented 3 algorithms in this report.

from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructType,StringType
import pyspark.sql.functions as f
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor, DecisionTreeRegressor, GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
# Name for Spark session
spark = SparkSession.builder.appName("COVID").getOrCreate()

In [0]:
# Importing libraries
covid_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/FileStore/tables/country_wise_latest-2.csv')
who_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/FileStore/tables/WHO_COVID_19_global_data-10.csv')
owid_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/FileStore/tables/owid_covid_data-9.csv')

In [0]:
covid_df.show()

+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
 Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase| WHO Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
 Afghanistan| 36263| 1269| 25198| 9796| 106| 10| 18| 3.5| 69.49| 5.04| 35526| 737| 2.07|Eastern Mediterra...|
 Albania| 4880| 144| 2745| 1991| 117| 6| 63| 2.95| 56.25| 5.25| 4171| 709| 17.0| Europe|
 Algeria| 27973| 1163| 18837| 7973| 616| 8| 749| 4.16| 67.34| 6.17| 23691| 4282| 18.07| Africa|
 Andorra| 907| 52| 803| 52| 10| 0| 0| 5.73| 88.53| 6.48| 884| 23| 2.6| Europe|
 Angola| 950| 41| 242| 667| 18| 1| 0| 4.32| 25.47| 16.94| 749| 201| 26.84| Africa|
Antigua and Barbuda| 86| 3| 65| 18| 4| 0| 5| 3.49| 75.58| 4.62| 76| 10| 13.16| Americas|
 Argentina| 167416| 3059| 72575| 91782| 4890| 120| 2057| 1.83| 43.35| 4.21| 130774| 36642| 28.02| Americas|
 Armenia| 37390| 711| 26665| 10014| 73| 6| 187| 1.9| 71.32| 2.67| 34981| 2409| 6.89| Europe|
 Australia| 15303| 167| 9311| 5825| 368| 6| 137| 1.09| 60.84| 1.79| 12428| 2875| 23.13| Western Pacific|
 Austria| 20558| 713| 18246| 1599| 86| 1| 37| 3.47| 88.75| 3.91| 19743| 815| 4.13| Europe|
 Azerbaijan| 30446| 423| 23242| 6781| 396| 6| 558| 1.39| 76.34| 1.82| 27890| 2556| 9.16| Europe|
 Bahamas| 382| 11| 91| 280| 40| 0| 0| 2.88| 23.82| 12.09| 174| 208| 119.54| Americas|
 Bahrain| 39482| 141| 36110| 3231| 351| 1| 421| 0.36| 91.46| 0.39| 36936| 2546| 6.89|Eastern Mediterra...|
 Bangladesh| 226225| 2965| 125683| 97577| 2772| 37| 1801| 1.31| 55.56| 2.36| 207453| 18772| 9.05| South-East Asia|
 Barbados| 110| 7| 94| 9| 0| 0| 0| 6.36| 85.45| 7.45| 106| 4| 3.77| Americas|
 Belarus| 67251| 538| 60492| 6221| 119| 4| 67| 0.8| 89.95| 0.89| 66213| 1038| 1.57| Europe|
 Belgium| 66428| 9822| 17452| 39154| 402| 1| 14| 14.79| 26.27| 56.28| 64094| 2334| 3.64| Europe|
 Belize| 48| 2| 26| 20| 0| 0| 0| 4.17| 54.17| 7.69| 40| 8| 20.0| Americas|
 Benin| 1770| 35| 1036| 699| 0| 0| 0| 1.98| 58.53| 3.38| 1602| 168| 10.49| Africa|
 Bhutan| 99| 0| 86| 13| 4| 0| 1| 0.0| 86.87| 0.0| 90| 9| 10.0| South-East Asia|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
only showing top 20 rows

In [0]:
who_df.show()

+-------------+------------+-----------+----------+---------+----------------+----------+-----------------+
Date_reported|Country_code| Country|WHO_region|New_cases|Cumulative_cases|New_deaths|Cumulative_deaths|
+-------------+------------+-----------+----------+---------+----------------+----------+-----------------+
 2020-01-03| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-04| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-05| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-06| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-07| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-08| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-09| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-10| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-11| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-12| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-13| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-14| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-15| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-16| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-17| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-18| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-19| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-20| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-21| AF|Afghanistan| EMRO| 0| 0| 0| 0|
 2020-01-22| AF|Afghanistan| EMRO| 0| 0| 0| 0|
+-------------+------------+-----------+----------+---------+----------------+----------+-----------------+
only showing top 20 rows

In [0]:
owid_df.show()

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+---------------+---------------------+-------------------+--------------+------------+----------------------+--------------------------+---------------+-----------------------+
iso_code|continent| location| date|total_cases|new_cases|new_cases_smoothed|total_deaths|new_deaths|new_deaths_smoothed|total_cases_per_million|new_cases_per_million|new_cases_smoothed_per_million|total_deaths_per_million|new_deaths_per_million|new_deaths_smoothed_per_million|reproduction_rate|icu_patients|icu_patients_per_million|hosp_patients|hosp_patients_per_million|weekly_icu_admissions|weekly_icu_admissions_per_million|weekly_hosp_admissions|weekly_hosp_admissions_per_million|new_tests|total_tests|total_tests_per_thousand|new_tests_per_thousand|new_tests_smoothed|new_tests_smoothed_per_thousand|positive_rate|tests_per_case|tests_units|total_vaccinations|people_vaccinated|people_fully_vaccinated|new_vaccinations|new_vaccinations_smoothed|total_vaccinations_per_hundred|people_vaccinated_per_hundred|people_fully_vaccinated_per_hundred|new_vaccinations_smoothed_per_million|stringency_index| population|population_density|median_age|aged_65_older|aged_70_older|gdp_per_capita|extreme_poverty|cardiovasc_death_rate|diabetes_prevalence|female_smokers|male_smokers|handwashing_facilities|hospital_beds_per_thousand|life_expectancy|human_development_index|
+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+---------------+---------------------+-------------------+--------------+------------+----------------------+--------------------------+---------------+-----------------------+
 AFG| Asia|Afghanistan|2020-02-24| 1.0| 1.0| null| null| null| null| 0.026| 0.026| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| 8.33|3.8928341E7| 54.422| 18.6| 2.581| 1.337| 1803.987| null| 597.029| 9.59| null| null| 37.746| 0.5| 64.83| 0.511|
 AFG| Asia|Afghanistan|2020-02-25| 1.0| 0.0| null| null| null| null| 0.026| 0.0| null| null| null|

In [0]:
covid_df.printSchema()

root
-- Country/Region: string (nullable = true)
-- Confirmed: integer (nullable = true)
-- Deaths: integer (nullable = true)
-- Recovered: integer (nullable = true)
-- Active: integer (nullable = true)
-- New cases: integer (nullable = true)
-- New deaths: integer (nullable = true)
-- New recovered: integer (nullable = true)
-- Deaths / 100 Cases: double (nullable = true)
-- Recovered / 100 Cases: double (nullable = true)
-- Deaths / 100 Recovered: string (nullable = true)
-- Confirmed last week: integer (nullable = true)
-- 1 week change: integer (nullable = true)
-- 1 week % increase: double (nullable = true)
-- WHO Region: string (nullable = true)

In [0]:
who_df.printSchema()

root
-- Date_reported: string (nullable = true)
-- Country_code: string (nullable = true)
-- Country: string (nullable = true)
-- WHO_region: string (nullable = true)
-- New_cases: integer (nullable = true)
-- Cumulative_cases: integer (nullable = true)
-- New_deaths: integer (nullable = true)
-- Cumulative_deaths: integer (nullable = true)

In [0]:
owid_df.printSchema()

root
-- iso_code: string (nullable = true)
-- continent: string (nullable = true)
-- location: string (nullable = true)
-- date: string (nullable = true)
-- total_cases: double (nullable = true)
-- new_cases: double (nullable = true)
-- new_cases_smoothed: double (nullable = true)
-- total_deaths: double (nullable = true)
-- new_deaths: double (nullable = true)
-- new_deaths_smoothed: double (nullable = true)
-- total_cases_per_million: double (nullable = true)
-- new_cases_per_million: double (nullable = true)
-- new_cases_smoothed_per_million: double (nullable = true)
-- total_deaths_per_million: double (nullable = true)
-- new_deaths_per_million: double (nullable = true)
-- new_deaths_smoothed_per_million: double (nullable = true)
-- reproduction_rate: double (nullable = true)
-- icu_patients: double (nullable = true)
-- icu_patients_per_million: double (nullable = true)
-- hosp_patients: double (nullable = true)
-- hosp_patients_per_million: double (nullable = true)
-- weekly_icu_admissions: double (nullable = true)
-- weekly_icu_admissions_per_million: double (nullable = true)
-- weekly_hosp_admissions: double (nullable = true)
-- weekly_hosp_admissions_per_million: double (nullable = true)
-- new_tests: double (nullable = true)
-- total_tests: double (nullable = true)
-- total_tests_per_thousand: double (nullable = true)
-- new_tests_per_thousand: double (nullable = true)
-- new_tests_smoothed: double (nullable = true)
-- new_tests_smoothed_per_thousand: double (nullable = true)
-- positive_rate: double (nullable = true)
-- tests_per_case: double (nullable = true)
-- tests_units: string (nullable = true)
-- total_vaccinations: double (nullable = true)
-- people_vaccinated: double (nullable = true)
-- people_fully_vaccinated: double (nullable = true)
-- new_vaccinations: double (nullable = true)
-- new_vaccinations_smoothed: double (nullable = true)
-- total_vaccinations_per_hundred: double (nullable = true)
-- people_vaccinated_per_hundred: double (nullable = true)
-- people_fully_vaccinated_per_hundred: double (nullable = true)
-- new_vaccinations_smoothed_per_million: double (nullable = true)
-- stringency_index: double (nullable = true)
-- population: double (nullable = true)
-- population_density: double (nullable = true)
-- median_age: double (nullable = true)
-- aged_65_older: double (nullable = true)
-- aged_70_older: double (nullable = true)
-- gdp_per_capita: double (nullable = true)
-- extreme_poverty: double (nullable = true)
-- cardiovasc_death_rate: double (nullable = true)
-- diabetes_prevalence: double (nullable = true)
-- female_smokers: double (nullable = true)
-- male_smokers: double (nullable = true)
-- handwashing_facilities: double (nullable = true)
-- hospital_beds_per_thousand: double (nullable = true)
-- life_expectancy: double (nullable = true)
-- human_development_index: double (nullable = true)

In [0]:
#Converting the datatype to Integer for implementing the algorithms

from pyspark.sql.types import IntegerType
covid_df = covid_df.withColumn("Recovered / 100 Cases", covid_df["Recovered / 100 Cases"].cast(IntegerType()))
covid_df = covid_df.withColumn("Deaths / 100 Recovered", covid_df["Deaths / 100 Recovered"].cast(IntegerType()))
covid_df = covid_df.withColumn("Deaths / 100 Cases", covid_df["Deaths / 100 Cases"].cast(IntegerType()))
covid_df = covid_df.withColumn("1 week % increase", covid_df["1 week % increase"].cast(IntegerType()))


In [0]:
# All the numerical columns are converted to integer type
covid_df.printSchema()

root
-- Country/Region: string (nullable = true)
-- Confirmed: integer (nullable = true)
-- Deaths: integer (nullable = true)
-- Recovered: integer (nullable = true)
-- Active: integer (nullable = true)
-- New cases: integer (nullable = true)
-- New deaths: integer (nullable = true)
-- New recovered: integer (nullable = true)
-- Deaths / 100 Cases: integer (nullable = true)
-- Recovered / 100 Cases: integer (nullable = true)
-- Deaths / 100 Recovered: integer (nullable = true)
-- Confirmed last week: integer (nullable = true)
-- 1 week change: integer (nullable = true)
-- 1 week % increase: integer (nullable = true)
-- WHO Region: string (nullable = true)

In [0]:
covid_df.describe().show()

+-------+--------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+-----------------+------------------+---------------+
summary|Country/Region| Confirmed| Deaths| Recovered| Active| New cases| New deaths| New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week| 1 week change| 1 week % increase| WHO Region|
+-------+--------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+-----------------+------------------+---------------+
 count| 187| 187| 187| 187| 187| 187| 187| 187| 187| 187| 182| 187| 187| 187| 187|
 mean| null| 88130.935828877| 3497.51871657754| 50631.48128342246| 34001.935828877|1222.957219251337| 28.9572192513369| 933.812834224599|2.5614973262032086| 64.33689839572193| 40.120879120879124| 78682.47593582887|9448.459893048128|13.181818181818182| null|
 stddev| null|383318.6638306154|14100.00248201848|190188.18964313966|213326.17337142891|5710.374790280563|120.03717295550823|4197.719634688254|3.4376495320349836| 26.225631331112456| 336.68738340972806| 338273.67656653706|47491.12768403526|24.512210933501787| null|
 min| Afghanistan| 10| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| -47| -3| Africa|
 max| Zimbabwe| 4290259| 148011| 1846641| 2816444| 56336| 1076| 33728| 28| 100| 3259| 3834677| 455582| 226|Western Pacific|
+-------+--------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+-----------------+------------------+---------------+

In [0]:
display(covid_df.select('Confirmed', 'Deaths'))

Confirmed,Deaths
36263,1269
4880,144
27973,1163
907,52
950,41
86,3
167416,3059
37390,711
15303,167
20558,713


In [0]:
display(who_df.select('Cumulative_cases', 'Cumulative_deaths'))

Cumulative_cases,Cumulative_deaths
0,0
0,0
0,0
0,0
0,0
0,0
0,0
0,0
0,0
0,0


In [0]:
display(owid_df.select('continent', 'total_cases'))

continent,total_cases
Asia,1.0
Asia,1.0
Asia,1.0
Asia,1.0
Asia,1.0
Asia,1.0
Asia,1.0
Asia,1.0
Asia,2.0
Asia,4.0


In [0]:
#Selecting the growth factor column to determine the increase of covid cases per week
covid_df.select("1 week change").show()

+-------------+
1 week change|
+-------------+
 737|
 709|
 4282|
 23|
 201|
 10|
 36642|
 2409|
 2875|
 815|
 2556|
 208|
 2546|
 18772|
 4|
 1038|
 2334|
 8|
 168|
 9|
+-------------+
only showing top 20 rows

In [0]:
covid_df = covid_df.withColumnRenamed("1 week change", "Change_Growth")

In [0]:
covid_df.show()

+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
 Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|Weekly Change|1 week % increase| WHO Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
 Afghanistan| 36263| 1269| 25198| 9796| 106| 10| 18| 3| 69| 5| 35526| 737| 2|Eastern Mediterra...|
 Albania| 4880| 144| 2745| 1991| 117| 6| 63| 2| 56| 5| 4171| 709| 17| Europe|
 Algeria| 27973| 1163| 18837| 7973| 616| 8| 749| 4| 67| 6| 23691| 4282| 18| Africa|
 Andorra| 907| 52| 803| 52| 10| 0| 0| 5| 88| 6| 884| 23| 2| Europe|
 Angola| 950| 41| 242| 667| 18| 1| 0| 4| 25| 16| 749| 201| 26| Africa|
Antigua and Barbuda| 86| 3| 65| 18| 4| 0| 5| 3| 75| 4| 76| 10| 13| Americas|
 Argentina| 167416| 3059| 72575| 91782| 4890| 120| 2057| 1| 43| 4| 130774| 36642| 28| Americas|
 Armenia| 37390| 711| 26665| 10014| 73| 6| 187| 1| 71| 2| 34981| 2409| 6| Europe|
 Australia| 15303| 167| 9311| 5825| 368| 6| 137| 1| 60| 1| 12428| 2875| 23| Western Pacific|
 Austria| 20558| 713| 18246| 1599| 86| 1| 37| 3| 88| 3| 19743| 815| 4| Europe|
 Azerbaijan| 30446| 423| 23242| 6781| 396| 6| 558| 1| 76| 1| 27890| 2556| 9| Europe|
 Bahamas| 382| 11| 91| 280| 40| 0| 0| 2| 23| 12| 174| 208| 119| Americas|
 Bahrain| 39482| 141| 36110| 3231| 351| 1| 421| 0| 91| 0| 36936| 2546| 6|Eastern Mediterra...|
 Bangladesh| 226225| 2965| 125683| 97577| 2772| 37| 1801| 1| 55| 2| 207453| 18772| 9| South-East Asia|
 Barbados| 110| 7| 94| 9| 0| 0| 0| 6| 85| 7| 106| 4| 3| Americas|
 Belarus| 67251| 538| 60492| 6221| 119| 4| 67| 0| 89| 0| 66213| 1038| 1| Europe|
 Belgium| 66428| 9822| 17452| 39154| 402| 1| 14| 14| 26| 56| 64094| 2334| 3| Europe|
 Belize| 48| 2| 26| 20| 0| 0| 0| 4| 54| 7| 40| 8| 20| Americas|
 Benin| 1770| 35| 1036| 699| 0| 0| 0| 1| 58| 3| 1602| 168| 10| Africa|
 Bhutan| 99| 0| 86| 13| 4| 0| 1| 0| 86| 0| 90| 9| 10| South-East Asia|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
only showing top 20 rows

In [0]:
covid_df = covid_df.na.fill("0", subset = ['New cases', 'New deaths', 'New recovered', 'Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered'])

In [0]:
covid_df.select("Change_Growth").show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3660196014832006> in <module> 
 ----> 1 covid_df . select ( "Change_Growth" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1437 [ Row ( name = u'Alice' , age = 12 ) , Row ( name = u'Bob' , age = 15 ) ] 
 1438 """
 -> 1439 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1440 return DataFrame ( jdf , self . sql_ctx ) 
 1441 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : cannot resolve '`Change_Growth`' given input columns: [1 week % increase, Active, Confirmed, Confirmed last week, Country/Region, Deaths, Deaths / 100 Cases, Deaths / 100 Recovered, New cases, New deaths, New recovered, Recovered, Recovered / 100 Cases, WHO Region, Weekly Change];;
'Project ['Change_Growth]
+- Project [Country/Region#490, Confirmed#491, Deaths#492, Recovered#493, Active#494, New cases#495, New deaths#496, New recovered#497, Deaths / 100 Cases#1131, Recovered / 100 Cases#1099, Deaths / 100 Recovered#1115, Confirmed last week#501, Weekly Change#3528, 1 week % increase#1147, WHO Region#504]
 +- Project [Country/Region#490, Confirmed#491, Deaths#492, Recovered#493, Active#494, New cases#495, New deaths#496, New recovered#497, Deaths / 100 Cases#1131, Recovered / 100 Cases#1099, Deaths / 100 Recovered#1115, Confirmed last week#501, 1 week change#502 AS Weekly Change#3528, 1 week % increase#1147, WHO Region#504]
 +- Project [Country/Region#490, Confirmed#491, Deaths#492, Recovered#493, Active#494, New cases#495, New deaths#496, New recovered#497, Deaths / 100 Cases#1131, Recovered / 100 Cases#1099, Deaths / 100 Recovered#1115, Confirmed last week#501, 1 week change#502, cast(1 week % increase#503 as int) AS 1 week % increase#1147, WHO Region#504]
 +- Project [Country/Region#490, Confirmed#491, Deaths#492, Recovered#493, Active#494, New cases#495, New deaths#496, New recovered#497, cast(Deaths / 100 Cases#498 as int) AS Deaths / 100 Cases#1131, Recovered / 100 Cases#1099, Deaths / 100 Recovered#1115, Confirmed last week#501, 1 week change#502, 1 week % increase#503, WHO Region#504]
 +- Project [Country/Region#490, Confirmed#491, Deaths#492, Recovered#493, Active#494, New cases#495, New deaths#496, New recovered#497, Deaths / 100 Cases#498, Recovered / 100 Cases#1099, cast(Deaths / 100 Recovered#500 as int) AS Deaths / 100 Recovered#1115, Confirmed last week#501, 1 week change#502, 1 week % increase#503, WHO Region#504]
 +- Project [Country/Region#490, Confirmed#491, Deaths#492, Recovered#493, Active#494, New cases#495, New deaths#496, New recovered#497, Deaths / 100 Cases#498, cast(Recovered / 100 Cases#499 as int) AS Recovered / 100 Cases#1099, Deaths / 100 Recovered#500, Confirmed last week#501, 1 week change#502, 1 week % increase#503, WHO Region#504]
 +- Relation[Country/Region#490,Confirmed#491,Deaths#492,Recovered#493,Active#494,New cases#495,New deaths#496,New recovered#497,Deaths / 100 Cases#498,Recovered / 100 Cases#499,Deaths / 100 Recovered#500,Confirmed last week#501,1 week change#502,1 week % increase#503,WHO Region#504] csv

In [0]:
#Filtering out the column growth factor
covid_df.filter("Change_Growth == 0").select(["Country/Region","New cases","Change_Growth" ]).show()

--------------------------------------------------------------------------- 
 ParseException Traceback (most recent call last)
 <command-3660196014832007> in <module> 
 1 #Filtering out the column growth factor 
 ----> 2 covid_df . filter ( "Weekly Change == 0" ) . select ( [ "Country/Region" , "New cases" , "Weekly Change" ] ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in filter (self, condition) 
 1475 """
 1476 if isinstance ( condition , basestring ) : 
 -> 1477 jdf = self . _jdf . filter ( condition ) 
 1478 elif isinstance ( condition , Column ) : 
 1479 jdf = self . _jdf . filter ( condition . _jc ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 ParseException : 
mismatched input '==' expecting {<EOF>, '-'}(line 1, pos 14)

== SQL ==
Weekly Change == 0
--------------^^^

In [0]:
covid_new = covid_df.filter(covid_df["Change_Growth"] > 0).sort("Change_Growth", ascending = True)

In [0]:
covid_new.select("Change_Growth").show()

+-------------+
Change_Growth|
+-------------+
 1|
 1|
 1|
 1|
 2|
 2|
 2|
 4|
 5|
 6|
 7|
 8|
 9|
 9|
 10|
 11|
 11|
 13|
 14|
 15|
+-------------+
only showing top 20 rows

In [0]:
covid_new.describe().show()

+-------+--------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+------------------+------------------+---------------+
summary|Country/Region| Confirmed| Deaths| Recovered| Active| New cases| New deaths| New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week| Change_Growth| 1 week % increase| WHO Region|
+-------+--------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+------------------+------------------+---------------+
 count| 174| 174| 174| 174| 174| 174| 174| 174| 174| 174| 169| 174| 174| 174| 174|
 mean| null|94682.02298850575| 3758.080459770115| 54396.84482758621|36527.097701149425|1314.2816091954023|31.120689655172413|1003.580459770115|2.6149425287356323| 63.58045977011494| 42.976331360946745| 84527.3735632184|10154.649425287356|14.183908045977011| null|
 stddev| null|396678.0303352999|14586.544569931519|196683.77697176125| 220987.4633903633| 5910.842358130151|124.19322223083086|4344.483640918166| 3.465103614650184| 25.61186204171881| 349.30502349159104| 350047.8326834572| 49169.84377589328| 25.12812427790025| null|
 min| Afghanistan| 14| 0| 0| 1| 0| 0| 0| 0| 0| 0| 13| 1| 0| Africa|
 max| Zimbabwe| 4290259| 148011| 1846641| 2816444| 56336| 1076| 33728| 28| 98| 3259| 3834677| 455582| 226|Western Pacific|
+-------+--------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+------------------+------------------+---------------+

In [0]:
covid_new.printSchema()

root
-- Country/Region: string (nullable = true)
-- Confirmed: integer (nullable = true)
-- Deaths: integer (nullable = true)
-- Recovered: integer (nullable = true)
-- Active: integer (nullable = true)
-- New cases: integer (nullable = true)
-- New deaths: integer (nullable = true)
-- New recovered: integer (nullable = true)
-- Deaths / 100 Cases: integer (nullable = true)
-- Recovered / 100 Cases: integer (nullable = true)
-- Deaths / 100 Recovered: integer (nullable = true)
-- Confirmed last week: integer (nullable = true)
-- Change_Growth: integer (nullable = true)
-- 1 week % increase: integer (nullable = true)
-- WHO Region: string (nullable = true)

In [0]:
covid_new.describe().toPandas().transpose()

Out[25]:

,0,1,2,3,4
summary,count,mean,stddev,min,max
Country/Region,174,None,None,Afghanistan,Zimbabwe
Confirmed,174,94682.02298850575,396678.0303352999,14,4290259
Deaths,174,3758.080459770115,14586.544569931519,0,148011
Recovered,174,54396.84482758621,196683.77697176125,0,1846641
Active,174,36527.097701149425,220987.4633903633,1,2816444
New cases,174,1314.2816091954023,5910.842358130151,0,56336
New deaths,174,31.120689655172413,124.19322223083086,0,1076
New recovered,174,1003.580459770115,4344.483640918166,0,33728
Deaths / 100 Cases,174,2.6149425287356323,3.465103614650184,0,28


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
numeric_features = [t[0] for t in covid_new.dtypes if t[1] == 'int' or t[1] == 'double']
covid_new = covid_new.select(numeric_features).sample(False, 0.8).toPandas()
axis = pd.plotting.scatter_matrix(covid_new, figsize=(20, 20))
n = len(covid_new.columns)
for i in range(n):
    v = axis[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axis[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

In [0]:
import six
for i in covid_df.columns:
    if not( isinstance(covid_new.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to New cases for ", i, covid_df.stat.corr('New cases',i))

Correlation to New cases for Confirmed 0.9097198996343834
Correlation to New cases for Deaths 0.8069745096857259
Correlation to New cases for Recovered 0.8189418469364926
Correlation to New cases for Active 0.8511904734280377
Correlation to New cases for New cases 1.0
Correlation to New cases for New deaths 0.9359465775223127
Correlation to New cases for New recovered 0.9147645967352609
Correlation to New cases for Deaths / 100 Cases 0.019529993388641843
Correlation to New cases for Recovered / 100 Cases -0.07957205299660208
Correlation to New cases for Deaths / 100 Recovered -0.011120675927808832
Correlation to New cases for Confirmed last week 0.8960835207253248
Correlation to New cases for Change_Growth 0.9599929827915403
Correlation to New cases for 1 week % increase 0.030558861020469168

In [0]:
# Preparing data for Machine Learning

from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['New cases', 'New deaths', 'New recovered', 'Change_Growth'], outputCol = 'features')
vector_df = vectorAssembler.transform(covid_new)
vector_df = covid_new.select(['New cases', 'New recovered'])
vector_df.show(50)

+---------+-------------+
New cases|New recovered|
+---------+-------------+
 0| 0|
 0| 0|
 1| 0|
 0| 0|
 1| 1|
 0| 0|
 1| 4|
 0| 0|
 0| 0|
 0| 0|
 0| 0|
 0| 0|
 0| 2|
 4| 1|
 4| 5|
 4| 0|
 1| 0|
 0| 1|
 2| 2|
 7| 0|
 0| 0|
 3| 0|
 10| 0|
 1| 0|
 0| 0|
 0| 0|
 7| 0|
 14| 6|
 3| 2|
 9| 11|
 0| 0|
 11| 0|
 11| 0|
 6| 2|
 0| 0|
 19| 0|
 1| 4|
 17| 22|
 5| 0|
 13| 4|
 5| 5|
 18| 22|
 11| 4|
 0| 4|
 10| 36|
 3| 15|
 23| 15|
 37| 2|
 6| 8|
 43| 0|
+---------+-------------+
only showing top 50 rows

In [0]:
#Avg of growthfactor used to predict the percentile of week 2
confirm_total = covid_new.groupBy("Confirmed").sum()
death_total = covid_new.groupBy("Deaths").sum()
recovered_total = covid_new.groupBy("Recovered").sum()

change_growth = covid_new.groupBy("Change_Growth").sum()

In [0]:
change_growth.show()

+-------------+--------------+-----------+--------------+-----------+--------------+---------------+------------------+-----------------------+--------------------------+---------------------------+------------------------+------------------+----------------------+
Change_Growth|sum(Confirmed)|sum(Deaths)|sum(Recovered)|sum(Active)|sum(New cases)|sum(New deaths)|sum(New recovered)|sum(Deaths / 100 Cases)|sum(Recovered / 100 Cases)|sum(Deaths / 100 Recovered)|sum(Confirmed last week)|sum(Change_Growth)|sum(1 week % increase)|
+-------------+--------------+-----------+--------------+-----------+--------------+---------------+------------------+-----------------------+--------------------------+---------------------------+------------------------+------------------+----------------------+
 737| 36263| 1269| 25198| 9796| 106| 10| 18| 3| 69| 5| 35526| 737| 2|
 4307| 15841| 115| 3824| 11902| 612| 11| 88| 0| 24| 3| 11534| 4307| 37|
 2888| 24141| 543| 0| 23598| 411| 9| 0| 2| 0| null| 21253| 2888| 13|
 756| 7189| 49| 4682| 2458| 205| 0| 219| 0| 65| 1| 6433| 756| 11|
 126| 25892| 1764| 23364| 764| 11| 0| 0| 6| 90| 7| 25766| 126| 0|
 847| 2827| 64| 577| 2186| 158| 4| 24| 2| 20| 11| 1980| 847| 42|
 1226| 4552| 140| 2815| 1597| 71| 1| 465| 3| 61| 4| 3326| 1226| 36|
 27| 2351| 100| 2072| 179| 0| 0| 0| 8| 175| 8| 2297| 54| 4|
 1418| 15516| 373| 11428| 3715| 192| 2| 0| 2| 73| 3| 14098| 1418| 10|
 44| 853| 10| 714| 129| 11| 0| 0| 1| 83| 1| 809| 44| 5|
 908| 18752| 48| 13754| 4950| 139| 3| 626| 0| 73| 0| 17844| 908| 5|
 2409| 37390| 711| 26665| 10014| 73| 6| 187| 1| 71| 2| 34981| 2409| 6|
 3325| 20887| 316| 16553| 4018| 609| 3| 115| 1| 79| 1| 17562| 3325| 18|
 91| 874| 18| 607| 249| 6| 0| 8| 2| 69| 2| 783| 91| 11|
 11982| 63985| 474| 27133| 36378| 2029| 4| 108| 0| 42| 1| 52003| 11982| 23|
 285| 6208| 156| 4653| 1399| 37| 0| 223| 2| 74| 3| 5923| 285| 4|
 3654| 15988| 146| 9959| 5883| 525| 4| 213| 0| 62| 1| 12334| 3654| 29|
 22| 1060| 19| 852| 189| 3| 0| 0| 1| 80| 2| 1038| 22| 2|
 2828| 15035| 408| 7778| 6849| 405| 8| 130| 2| 51| 5| 12207| 2828| 23|
 8658| 77058| 393| 57028| 19637| 1053| 9| 1729| 0| 74| 0| 68400| 8658| 12|
+-------------+--------------+-----------+--------------+-----------+--------------+---------------+------------------+-----------------------+--------------------------+---------------------------+------------------------+------------------+----------------------+
only showing top 20 rows

In [0]:
vec_assm = VectorAssembler(inputCols=['Confirmed','Change_Growth', 'Confirmed last week'], outputCol='features')

In [0]:
who_vec_assm = VectorAssembler(inputCols=['New_cases', 'New_deaths', 'Cumulative_cases', 'Cumulative_deaths', outputCol='features'])

In [0]:
new_vec = vec_assm.transform(covid_new)

In [0]:
new_vec.printSchema()

root
-- Country/Region: string (nullable = true)
-- Confirmed: integer (nullable = true)
-- Deaths: integer (nullable = true)
-- Recovered: integer (nullable = true)
-- Active: integer (nullable = true)
-- New cases: integer (nullable = true)
-- New deaths: integer (nullable = true)
-- New recovered: integer (nullable = true)
-- Deaths / 100 Cases: integer (nullable = true)
-- Recovered / 100 Cases: integer (nullable = true)
-- Deaths / 100 Recovered: integer (nullable = true)
-- Confirmed last week: integer (nullable = true)
-- Change_Growth: integer (nullable = true)
-- 1 week % increase: integer (nullable = true)
-- WHO Region: string (nullable = true)
-- features: vector (nullable = true)

In [0]:
vec_f = new_vec.select("features", "Change_Growth")

In [0]:
vec_f.show()

+--------------------+-------------+
 features|Change_Growth|
+--------------------+-------------+
 [14.0,1.0,13.0]| 1|
 [20.0,1.0,19.0]| 1|
 [344.0,1.0,343.0]| 1|
 [24.0,1.0,23.0]| 1|
 [289.0,2.0,287.0]| 2|
 [1557.0,2.0,1555.0]| 2|
 [52.0,2.0,50.0]| 2|
 [110.0,4.0,106.0]| 4|
 [1954.0,5.0,1949.0]| 5|
 [114.0,6.0,108.0]| 6|
 [116.0,7.0,109.0]| 7|
 [48.0,8.0,40.0]| 8|
 [99.0,9.0,90.0]| 9|
 [350.0,9.0,341.0]| 9|
 [86.0,10.0,76.0]| 10|
 [462.0,11.0,451.0]| 11|
 [148.0,11.0,137.0]| 11|
[2034.0,13.0,2021.0]| 13|
 [265.0,14.0,251.0]| 14|
[1854.0,15.0,1839.0]| 15|
+--------------------+-------------+
only showing top 20 rows

In [0]:
#Splitting the Dataset
train_X,test_X = vec_f.randomSplit([0.8,0.2])

In [0]:
train_X.show()

+-------------------+-------------+
 features|Change_Growth|
+-------------------+-------------+
 [14.0,1.0,13.0]| 1|
 [20.0,1.0,19.0]| 1|
 [24.0,1.0,23.0]| 1|
 [48.0,8.0,40.0]| 8|
 [52.0,2.0,50.0]| 2|
 [62.0,43.0,19.0]| 43|
 [99.0,9.0,90.0]| 9|
 [110.0,4.0,106.0]| 4|
 [114.0,6.0,108.0]| 6|
 [116.0,7.0,109.0]| 7|
 [265.0,14.0,251.0]| 14|
 [289.0,2.0,287.0]| 2|
[326.0,214.0,112.0]| 214|
 [344.0,1.0,343.0]| 1|
 [350.0,9.0,341.0]| 9|
 [354.0,20.0,334.0]| 20|
 [378.0,56.0,322.0]| 56|
[382.0,208.0,174.0]| 208|
 [389.0,52.0,337.0]| 52|
 [431.0,47.0,384.0]| 47|
+-------------------+-------------+
only showing top 20 rows

In [0]:
test_X.show()

+--------------------+-------------+
 features|Change_Growth|
+--------------------+-------------+
 [86.0,10.0,76.0]| 10|
 [148.0,11.0,137.0]| 11|
 [226.0,55.0,171.0]| 55|
 [701.0,24.0,677.0]| 24|
 [950.0,201.0,749.0]| 201|
[1167.0,60.0,1107.0]| 60|
[1455.0,74.0,1381.0]| 74|
 [1557.0,2.0,1555.0]| 2|
[2328.0,257.0,207...| 257|
[2532.0,86.0,2446.0]| 86|
[2805.0,75.0,2730.0]| 75|
[3200.0,349.0,285...| 349|
[7055.0,465.0,659...| 465|
[7340.0,287.0,705...| 287|
[7413.0,1536.0,58...| 1536|
[9690.0,2537.0,71...| 2537|
[10213.0,964.0,92...| 964|
[10621.0,1692.0,8...| 1692|
[14203.0,387.0,13...| 387|
[25892.0,126.0,25...| 126|
+--------------------+-------------+
only showing top 20 rows

In [0]:
# Implementing Linear Regression Model on the smallest dataset
lin_reg = LinearRegression(labelCol="Change_Growth")

In [0]:
lin_model = lin_reg.fit(train_X)

In [0]:
res = lin_model.evaluate(test_X)

In [0]:
res.residuals.show()

+--------------------+
 residuals|
+--------------------+
-2.62852626917720...|
-2.62134759889676...|
-2.65491054136646...|
-2.56138502940928E-5|
-2.70494041387792...|
-2.53581461677754...|
-2.51179103401000...|
-2.42659040106119...|
-2.57904610521109...|
-2.38173818729592...|
-2.33477890390076...|
-2.55574763627919...|
-2.16303295701436...|
-1.94806871718355...|
-3.18312515901197...|
-3.88042330996540...|
-2.24389943923597...|
-2.91558451408491...|
-1.14277964939901...|
6.586094343674631E-6|
+--------------------+
only showing top 20 rows

In [0]:
#Rootmeansquared error
res.rootMeanSquaredError

Out[45]: 5.858056487295799e-05

In [0]:
#Returning the coefficient of determination
res.r2

Out[46]: 1.0

In [0]:
res.describe().show()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3660196014832036> in <module> 
 ----> 1 res . describe ( ) 

 AttributeError : 'LinearRegressionSummary' object has no attribute 'describe'

In [0]:
#Data prediction using the Feature column 
feat_col = test_X.select("features")

In [0]:
feat_col.show()

+--------------------+
 features|
+--------------------+
 [86.0,10.0,76.0]|
 [148.0,11.0,137.0]|
 [226.0,55.0,171.0]|
 [701.0,24.0,677.0]|
 [950.0,201.0,749.0]|
[1167.0,60.0,1107.0]|
[1455.0,74.0,1381.0]|
 [1557.0,2.0,1555.0]|
[2328.0,257.0,207...|
[2532.0,86.0,2446.0]|
[2805.0,75.0,2730.0]|
[3200.0,349.0,285...|
[7055.0,465.0,659...|
[7340.0,287.0,705...|
[7413.0,1536.0,58...|
[9690.0,2537.0,71...|
[10213.0,964.0,92...|
[10621.0,1692.0,8...|
[14203.0,387.0,13...|
[25892.0,126.0,25...|
+--------------------+
only showing top 20 rows

In [0]:
lin_pred = lin_model.transform(feat_col)

In [0]:
# Growth Factor prediction
lin_pred.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
 [86.0,10.0,76.0]|10.000026285262692|
 [148.0,11.0,137.0]|11.000026213475989|
 [226.0,55.0,171.0]|55.000026549105414|
 [701.0,24.0,677.0]|24.000025613850294|
 [950.0,201.0,749.0]|201.00002704940414|
[1167.0,60.0,1107.0]| 60.00002535814617|
[1455.0,74.0,1381.0]| 74.00002511791034|
 [1557.0,2.0,1555.0]|2.0000242659040106|
[2328.0,257.0,207...|257.00002579046105|
[2532.0,86.0,2446.0]| 86.00002381738187|
[2805.0,75.0,2730.0]| 75.00002334778904|
[3200.0,349.0,285...|349.00002555747636|
[7055.0,465.0,659...|465.00002163032957|
[7340.0,287.0,705...| 287.0000194806872|
[7413.0,1536.0,58...|1536.0000318312516|
[9690.0,2537.0,71...| 2537.000038804233|
[10213.0,964.0,92...| 964.0000224389944|
[10621.0,1692.0,8...|1692.0000291558451|
[14203.0,387.0,13...| 387.0000114277965|
[25892.0,126.0,25...|125.99999341390566|
+--------------------+------------------+
only showing top 20 rows

In [0]:
# Decision Tree Implementation 

from pyspark.ml.regression import DecisionTreeRegressor
decision_tree = DecisionTreeRegressor(labelCol= "Change_Growth",featuresCol= "features")

In [0]:
decision_tree_model = decision_tree.fit(train_X)

In [0]:
decision_tree_prediction = decision_tree_model.transform(test_X)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluate_dt = RegressionEvaluator(labelCol= "Change_Growth")

In [0]:
pred_dt = evaluate_dt.evaluate(decision_tree_prediction)
print(pred_dt)

4473.1320624321115

In [0]:
# Random Forest Implementation
from pyspark.ml.regression import RandomForestRegressor

random_forest = RandomForestRegressor(numTrees= 100,labelCol= "Change_Growth", featuresCol= "features")
random_forest_model = random_forest.fit(train_X)
random_forest_prediction = random_forest_model.transform(test_X)

In [0]:
pred_rf = reg_eval.evaluate(random_forest_prediction)
print(pred_rf)

5032.201939557669

## The Linear Regression Model is working the best among all three.

## COVID_PROJECT ends here.